In [1]:
# Use GAN to generate synthesized characters on MNIST dataset

In [2]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image
from torchsummary import summary as PyTorchSummary

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [4]:
latent_size = 10
hidden_size1 = 256
hidden_size2 = 128
image_size = 28*28
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

In [5]:
# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [6]:
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = (0.5,0.5,0.5),
                                     std = (0.5, 0.5, 0.5))])

In [7]:
# MNIST dataset

In [8]:
mnist = torchvision.datasets.MNIST(root='./../data',
                                          train = True,
                                          transform = transform,
                                          download = True)

In [9]:
data_loader = torch.utils.data.DataLoader(dataset = mnist,
                                          batch_size = batch_size,
                                          shuffle = True)

## Build model

In [10]:
# Discriminator: a simple classification model

In [11]:
D = nn.Sequential(
    nn.Linear(image_size, hidden_size1),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size1, hidden_size2),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size2, 1),
    nn.Sigmoid())

In [12]:
PyTorchSummary(D, input_size = (1,image_size))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 256]         200,960
         LeakyReLU-2               [-1, 1, 256]               0
            Linear-3               [-1, 1, 128]          32,896
         LeakyReLU-4               [-1, 1, 128]               0
            Linear-5                 [-1, 1, 1]             129
           Sigmoid-6                 [-1, 1, 1]               0
Total params: 233,985
Trainable params: 233,985
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.89
Estimated Total Size (MB): 0.90
----------------------------------------------------------------


In [13]:
# Generator: like an autoencoder

In [14]:
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size2),
    nn.ReLU(),
    nn.Linear(hidden_size2, hidden_size1),
    nn.ReLU(),
    nn.Linear(hidden_size1, image_size),
    nn.Tanh())

In [15]:
PyTorchSummary(G, input_size = (1, latent_size))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 128]           1,408
              ReLU-2               [-1, 1, 128]               0
            Linear-3               [-1, 1, 256]          33,024
              ReLU-4               [-1, 1, 256]               0
            Linear-5               [-1, 1, 784]         201,488
              Tanh-6               [-1, 1, 784]               0
Total params: 235,920
Trainable params: 235,920
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 0.90
Estimated Total Size (MB): 0.92
----------------------------------------------------------------


In [16]:
# Device setting
D = D.to(device)
G = G.to(device)


In [17]:
# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0004)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0004)

In [18]:
def denorm(x):
    out = (x+1)/2
    return out.clamp(0, 1)

In [19]:
def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

## Training

In [20]:
total_step = len(data_loader)

In [21]:
emb = nn.Embedding(10, 10)
emb.weight.data = torch.eye(10)

In [22]:
for epoch in range(num_epochs):
    for i, (images,label) in enumerate(data_loader):
        images = images.reshape(batch_size,-1).to(device)
        
        #create labels with are later used as input for BCE loss
        real_labels = torch.ones(batch_size,1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)
        
        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #
        
        #compute BCE_loss using real images where BCE_loss(x,y) = -y*log(D(x)) - (1-y)*log(1-D(x))
        #second term of the loss is always zero for real cases because y=1 for real case
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        #compute BCEloss using fake images
        #First term of the loss is always zero for fake images
        
        #z = torch.randn(batch_size, latent_size).to(device)
        z = emb(label)#torch.LongTensor([[1, 2], [3, 4]]))
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        #backprop and optimize
        d_loss = 0.2*(d_loss_real + d_loss_fake)
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        #z = torch.randn(batch_size, latent_size).to(device)
        z = emb(label)#torch.LongTensor([[1, 2], [3, 4]]))
        fake_images = G(z)
        outputs = D(fake_images)
        
        #train G to maximize log(D(G(z))) instead of minimizing log(1-D(G(z)))
        #use real label to keep the first term so loss = -log(D(G(z)))
        g_loss = criterion(outputs, real_labels)
        
        #back_prop
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
            
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
        
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))
    

Epoch [0/200], Step [200/600], d_loss: 0.0194, g_loss: 2.6882, D(x): 1.00, D(G(z)): 0.09
Epoch [0/200], Step [400/600], d_loss: 0.0135, g_loss: 5.2135, D(x): 0.98, D(G(z)): 0.02
Epoch [0/200], Step [600/600], d_loss: 0.0023, g_loss: 6.3304, D(x): 1.00, D(G(z)): 0.01
Epoch [1/200], Step [200/600], d_loss: 0.0248, g_loss: 7.6412, D(x): 0.96, D(G(z)): 0.07
Epoch [1/200], Step [400/600], d_loss: 0.0161, g_loss: 5.4942, D(x): 0.99, D(G(z)): 0.06
Epoch [1/200], Step [600/600], d_loss: 0.0094, g_loss: 4.7781, D(x): 0.99, D(G(z)): 0.03
Epoch [2/200], Step [200/600], d_loss: 0.0182, g_loss: 4.2543, D(x): 0.97, D(G(z)): 0.05
Epoch [2/200], Step [400/600], d_loss: 0.0154, g_loss: 4.0169, D(x): 0.98, D(G(z)): 0.05
Epoch [2/200], Step [600/600], d_loss: 0.0128, g_loss: 5.5027, D(x): 0.96, D(G(z)): 0.01
Epoch [3/200], Step [200/600], d_loss: 0.0277, g_loss: 5.6463, D(x): 0.94, D(G(z)): 0.03
Epoch [3/200], Step [400/600], d_loss: 0.0265, g_loss: 3.5930, D(x): 0.96, D(G(z)): 0.03
Epoch [3/200], Step [

Epoch [30/200], Step [400/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [30/200], Step [600/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [31/200], Step [200/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [31/200], Step [400/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [31/200], Step [600/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [32/200], Step [200/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [32/200], Step [400/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [32/200], Step [600/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [33/200], Step [200/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [33/200], Step [400/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [33/200], Step [600/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(

Epoch [60/200], Step [400/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [60/200], Step [600/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [61/200], Step [200/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [61/200], Step [400/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [61/200], Step [600/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [62/200], Step [200/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [62/200], Step [400/600], d_loss: 0.0092, g_loss: 9.3746, D(x): 0.99, D(G(z)): 0.02
Epoch [62/200], Step [600/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [63/200], Step [200/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [63/200], Step [400/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [63/200], Step [600/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(

Epoch [90/200], Step [400/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [90/200], Step [600/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [91/200], Step [200/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [91/200], Step [400/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [91/200], Step [600/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [92/200], Step [200/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [92/200], Step [400/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [92/200], Step [600/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [93/200], Step [200/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [93/200], Step [400/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [93/200], Step [600/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(

Epoch [120/200], Step [200/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [120/200], Step [400/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [120/200], Step [600/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [121/200], Step [200/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [121/200], Step [400/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [121/200], Step [600/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [122/200], Step [200/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [122/200], Step [400/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [122/200], Step [600/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [123/200], Step [200/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [123/200], Step [400/600], d_loss: -0.0000, g_loss: 27.6310, D(x

Epoch [149/200], Step [600/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [150/200], Step [200/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [150/200], Step [400/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [150/200], Step [600/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [151/200], Step [200/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [151/200], Step [400/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [151/200], Step [600/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [152/200], Step [200/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [152/200], Step [400/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [152/200], Step [600/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [153/200], Step [200/600], d_loss: -0.0000, g_loss: 27.6310, D(x

Epoch [179/200], Step [400/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [179/200], Step [600/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [180/200], Step [200/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [180/200], Step [400/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [180/200], Step [600/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [181/200], Step [200/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [181/200], Step [400/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [181/200], Step [600/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [182/200], Step [200/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [182/200], Step [400/600], d_loss: -0.0000, g_loss: 27.6310, D(x): 1.00, D(G(z)): 0.00
Epoch [182/200], Step [600/600], d_loss: -0.0000, g_loss: 27.6310, D(x